In [25]:
from datasets import load_dataset

ds = load_dataset("darkQibit/russian-spam-detection")

In [26]:
import torch

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [27]:
print(ds)
print(ds['train'][0])
print(ds['train'].features)

DatasetDict({
    train: Dataset({
        features: ['message', 'label'],
        num_rows: 4511513
    })
})
{'message': 'Привет чтото подобное делал если через usb использовать нужно тот, у которого вендор предоставляет драйвера желательно с примером на андроиде', 'label': 0}
{'message': Value('string'), 'label': Value('int64')}


In [28]:
# Подсчет количества примеров:

# label = 1
count_label_1 = len(ds['train'].filter(lambda example: example['label'] == 1))
print(f"Количество примеров с label=1: {count_label_1}")

# label = 0
count_label_0 = len(ds['train'].filter(lambda example: example['label'] == 0))
print(f"Количество примеров с label=0: {count_label_0}")

# none or bad
count_label_bad = len(ds['train'].filter(
    lambda example: example['label'] is None or
                   (isinstance(example['label'], float) and np.isnan(example['label'])) or
                   example['label'] == ''
))
print(f"Количество примеров с label=none: {count_label_bad}")

Количество примеров с label=1: 347040
Количество примеров с label=0: 4164473
Количество примеров с label=none: 0


In [ ]:
# Перетасовка:
# labels_1 = ds['train'].filter(lambda example: example['label'] == 1)
# labels_0 = ds['train'].filter(lambda example: example['label'] == 0).shuffle(seed=42).select(range(len(labels_1)))

# print(f"Количество примеров с label=1: {len(labels_1)}")
# print(f"Количество примеров с label=0: {len(labels_0)}")

In [30]:
labels_1 = ds['train'].filter(lambda example: example['label'] == 1).select(range(22000))
labels_0 = ds['train'].filter(lambda example: example['label'] == 0).select(range(len(labels_1)))

print(f"Количество примеров с label=1: {len(labels_1)}")
print(f"Количество примеров с label=0: {len(labels_0)}")

Количество примеров с label=1: 22000
Количество примеров с label=0: 22000


In [31]:
from datasets import concatenate_datasets

dataset = concatenate_datasets([labels_1, labels_0])

dataset = dataset.shuffle(seed=42)

print(f"Размер объединенного датасета: {len(dataset)}")
print(f"Примеры: {dataset[0:5]}")

Размер объединенного датасета: 44000
Примеры: {'message': ['Нужны девушки для имиджевой съемки по бартеру 2122 февраля в районе не далеко от burj khalifa станции метро важно для съемке нужно будет сделать процедуры в салоне маникюр педикюр брови салон находиться в районе business bay если вас заинтересовала напишите пожалуйста в лс я скину примерные референсы и обсудим детали', 'Нужны срочнo люди на xaлтуpy нa пoл дня плaчy 10тысяч сразy поcле cмeны', 'А я про тбокс узнал уже только после покупки он никак не влиял на моё решение', 'Ищeм сотрyдников для paзбopки стapых кoнстрyкций на cтpойке уcлoвия 4000 pублей врeмя 7 чaсов в день зaдачи дeмoнтаж и вывоз мусopa бeз oпытa пишите в лc', 'А я хочу что все новое и актуальное было'], 'label': [1, 1, 0, 1, 0]}


<h2>Подготовка модели:</h2>

In [32]:
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import f1_score
import numpy as np

In [33]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Бинарная классификация
)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

In [34]:
# Токенизация
def tokenize(batch):
    tokenized = tokenizer(batch["message"], padding=True, truncation=True, max_length=512)
    tokenized["labels"] = [int(label) for label in batch["label"]]
    return tokenized

dataset = dataset.map(tokenize, batched=True)

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Создание data collator для динамического паддинга
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/44000 [00:00<?, ? examples/s]

In [37]:
# Настройка обучения
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    report_to="none",
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch",
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "f1_spam": f1_score(p.label_ids, preds, pos_label=1),  # F1 для токсичных
        "f1_non_spam": f1_score(p.label_ids, preds, pos_label=0),  # F1 для нетоксичных
        "accuracy": (preds == p.label_ids).mean(),
    }

# Создание и запуск Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2458030659.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Spam,F1 Non Spam,Accuracy
1,0.068800,0.056539,0.982284,0.983149,0.982727
2,0.046900,0.059481,0.983656,0.984503,0.984091
3,0.033700,0.056816,0.984741,0.985469,0.985114


TrainOutput(global_step=3300, training_loss=0.05172981348904696, metrics={'train_runtime': 853.0584, 'train_samples_per_second': 123.79, 'train_steps_per_second': 3.868, 'total_flos': 778716256665600.0, 'train_loss': 0.05172981348904696, 'epoch': 3.0})

In [ ]:
def predict_spam(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0]
    return {"spam": probs[1].item(), "non-spam": probs[0].item()}

In [ ]:
print("-"*70)
print("1)", predict_spam("Начислено на счёт +3 000 бонусов!"))
print("2)", predict_spam("Бонусы исчезнут через 2 дня ⏰"))
print("3)", predict_spam("Уважаемый клиент СбeрБанка! Ваша карта **** 4891 была заблокирована из-за попытки несанкционированного доступа. Для разблокировки перейдите по ссылке: sberbank-verify.ru/secure"))
print("4)", predict_spam("Алиэкспресс: Ваш заказ №RU789456 не может быть доставлен из-за проблем с таможней. Требуется оплата сбора 350 руб.: aliexpress-customs.ru/pay"))
print("5)", predict_spam("Начни зарабатывать от 80 000 руб. в месяц без опыта! Удалённая работа в международной компании. Обучение бесплатно. Регистрация: global-career.net"))
print("6)", predict_spam("Инвестируй в биткоин! Наша платформа гарантирует доход 5% в день. Минимальный депозит 5000 руб. Регистрация: btc-invest-2024.com"))
print("-"*70)
print("7)", predict_spam("Доброе утро, команда. Напоминаю, что сегодня в 11:00 состоится еженедельный стендап. Повестка: обсуждение спринта №24 и план на следующую неделю. Ссылка на Zoom в календаре."))
print("8)", predict_spam("Напоминание: Завтра в 15:00 у вас запланирована встреча со стоматологом в клинике 'Здоровье' (ул. Зелёная, 5). Пожалуйста, не опаздывайте."))
print("9)", predict_spam("Яндекс.Такси: Ваш заказ № 45678 подтверждён. Водитель Александр на Hyundai Solaris, госномер А123БВ45. Прибудет через 7 мин. Стоимость: 350 руб."))
print("10)", predict_spam("Привет! Как насчёт встретиться завтра в 18:00 в кофейне на Ленина? Нужно обсудить проект."))
print("11)", predict_spam("Привет! Не забыл, что завтра собираемся у Макса на день рождения? Начало в 19:00, адрес: ул. Мира, 15, кв. 42. Приносить ничего не нужно."))
print("12)", predict_spam("Уважаемые родители! Родительское собрание для 10 «Б» класса состоится 20 января в 18:00 в кабинете 205. Повестка: подготовка к ЕГЭ и результаты зимней сессии."))
print("-"*70)

{'spam': 0.0030265888199210167, 'non-spam': 0.9969733953475952}


In [53]:
tokenizer.save_pretrained("./spam_model")
model.save_pretrained("./spam_model")

In [55]:
!zip -r spam_model.zip ./spam_model/

  adding: spam_model/ (stored 0%)
  adding: spam_model/vocab.txt (deflated 64%)
  adding: spam_model/special_tokens_map.json (deflated 80%)
  adding: spam_model/config.json (deflated 49%)
  adding: spam_model/tokenizer_config.json (deflated 75%)
  adding: spam_model/model.safetensors (deflated 8%)
